## Import required libraries

In [45]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler


### Load the Dataset

In [93]:
df = pd.read_csv("01_Auto_Insurance_Fraud_Claims_Build_Test_Model_F01.csv")

df.head()



,Fraud_Ind,Policy_Start_Date,Policy_Expiry_Date,Accident_Date,Claim_Initiated_Date,Household_Num,Driver_Num,Policy_Num,Claim_Num,License_Plate,...,Commute_Discount,Claims_Status,Claim_Amount,Model_Year,Make,Model,Color,Police_Report,Initial_Odometer,Odometer_At_Incident
0,0,1/27/2025,7/28/2025,3/9/2025,4/8/2025,4670185276,SW059585,355924169,DZW11767,MGN62082,...,1,1,3954,2022,Volkswagen,Jetta,Red,No,87817,93539
1,0,6/1/2025,11/30/2025,6/4/2025,7/5/2025,8526791677,ZO779075,848558311,CVG38057,WXK26533,...,0,4,17917,2011,Toyota,Camry,Grey,Yes,22928,30075
2,0,6/22/2025,12/21/2025,7/4/2025,7/6/2025,4618820582,LG406912,473829217,RUO86042,NNL54173,...,0,3,13884,2015,Audi,A6,Black,No,79831,91252
3,0,1/2/2025,7/3/2025,1/27/2025,2/27/2025,3846198554,NJ146743,116719734,WOS87550,THC81673,...,0,3,4079,2015,BMW,3 Series,White,No,112017,117688
4,0,10/25/2024,4/25/2025,11/19/2024,12/22/2024,9856770505,KX673536,481900450,ACJ85041,NKY26268,...,1,2,3671,2020,Honda,Accord,Silver,No,101637,110535


### Basic data understanding

In [94]:
df.shape

(100000, 26)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Fraud_Ind                100000 non-null  int64 
 1   Policy_Start_Date        100000 non-null  object
 2   Policy_Expiry_Date       100000 non-null  object
 3   Accident_Date            100000 non-null  object
 4   Claim_Initiated_Date     100000 non-null  object
 5   Household_Num            100000 non-null  int64 
 6   Driver_Num               100000 non-null  object
 7   Policy_Num               100000 non-null  int64 
 8   Claim_Num                100000 non-null  object
 9   License_Plate            100000 non-null  object
 10  Drivers_License_Num      100000 non-null  object
 11  Drivers_License_State    100000 non-null  object
 12  DL_Expiry_Date           100000 non-null  object
 13  Date_At_Current_Address  100000 non-null  object
 14  Age                  

In [96]:
df.describe(include='all')

,Fraud_Ind,Policy_Start_Date,Policy_Expiry_Date,Accident_Date,Claim_Initiated_Date,Household_Num,Driver_Num,Policy_Num,Claim_Num,License_Plate,...,Commute_Discount,Claims_Status,Claim_Amount,Model_Year,Make,Model,Color,Police_Report,Initial_Odometer,Odometer_At_Incident
count,100000.000000,100000,100000,100000,100000,1.000000e+05,100000,1.000000e+05,100000,100000,...,100000.000000,100000.000000,100000.00000,100000.000000,100000,100000,100000,100000,100000.00000,100000.000000
unique,NaN,365,365,571,589,NaN,99992,NaN,99995,100000,...,NaN,NaN,NaN,NaN,10,38,8,2,NaN,NaN
top,NaN,1/20/2025,7/21/2025,2/8/2025,4/4/2025,NaN,XL973668,NaN,JRB45308,MGN62082,...,NaN,NaN,NaN,NaN,Toyota,Seltos,Black,Yes,NaN,NaN
freq,NaN,326,326,328,314,NaN,2,NaN,2,1,...,NaN,NaN,NaN,NaN,10138,3389,12597,50098,NaN,NaN
mean,0.074070,NaN,NaN,NaN,NaN,5.511925e+09,NaN,5.480008e+08,NaN,NaN,...,0.502340,2.499660,13019.77444,2017.504000,NaN,NaN,NaN,NaN,62416.42008,69419.199970
std,0.261886,NaN,NaN,NaN,NaN,2.594574e+09,NaN,2.602975e+08,NaN,NaN,...,0.499997,1.119389,6937.05472,4.610574,NaN,NaN,NaN,NaN,33295.18949,33428.992495
min,0.000000,NaN,NaN,NaN,NaN,1.000275e+09,NaN,1.000038e+08,NaN,NaN,...,0.000000,1.000000,1000.00000,2010.000000,NaN,NaN,NaN,NaN,5002.00000,7130.000000
25%,0.000000,NaN,NaN,NaN,NaN,3.269090e+09,NaN,3.224573e+08,NaN,NaN,...,0.000000,1.000000,6994.75000,2014.000000,NaN,NaN,NaN,NaN,33521.00000,40534.750000
50%,0.000000,NaN,NaN,NaN,NaN,5.525919e+09,NaN,5.474760e+08,NaN,NaN,...,1.000000,2.000000,13018.00000,2018.000000,NaN,NaN,NaN,NaN,62413.50000,69403.500000
75%,0.000000,NaN,NaN,NaN,NaN,7.752506e+09,NaN,7.738752e+08,NaN,NaN,...,1.000000,4.000000,19038.00000,2022.000000,NaN,NaN,NaN,NaN,91322.50000,98370.250000


# STEP:1 Missing Value Treatment

### Identify Missing Value Patterns (MCAR, MAR, MNAR)

#### Check missing values

In [8]:
df.isnull().sum().sort_values(ascending=False)

Fraud_Ind                  0
Policy_Start_Date          0
Initial_Odometer           0
Police_Report              0
Color                      0
Model                      0
Make                       0
Model_Year                 0
Claim_Amount               0
Claims_Status              0
Commute_Discount           0
Gender                     0
Age                        0
Date_At_Current_Address    0
DL_Expiry_Date             0
Drivers_License_State      0
Drivers_License_Num        0
License_Plate              0
Claim_Num                  0
Policy_Num                 0
Driver_Num                 0
Household_Num              0
Claim_Initiated_Date       0
Accident_Date              0
Policy_Expiry_Date         0
Odometer_At_Incident       0
dtype: int64

#### Missing value percentage

In [97]:
(df.isnull().mean() * 100).sort_values(ascending=False)


Fraud_Ind                  0.0
Policy_Start_Date          0.0
Initial_Odometer           0.0
Police_Report              0.0
Color                      0.0
Model                      0.0
Make                       0.0
Model_Year                 0.0
Claim_Amount               0.0
Claims_Status              0.0
Commute_Discount           0.0
Gender                     0.0
Age                        0.0
Date_At_Current_Address    0.0
DL_Expiry_Date             0.0
Drivers_License_State      0.0
Drivers_License_Num        0.0
License_Plate              0.0
Claim_Num                  0.0
Policy_Num                 0.0
Driver_Num                 0.0
Household_Num              0.0
Claim_Initiated_Date       0.0
Accident_Date              0.0
Policy_Expiry_Date         0.0
Odometer_At_Incident       0.0
dtype: float64

### Apply Imputation

#### Numerical → Median

In [52]:
num_cols = df.select_dtypes(include=['int64','float64']).columns

num_imputer = SimpleImputer(strategy='median')
df[num_cols] = num_imputer.fit_transform(df[num_cols])


#### Categorical → Mode

In [53]:
cat_cols = df.select_dtypes(include=['object']).columns

cat_imputer = SimpleImputer(strategy='most_frequent')
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])


# STEP 2: Outlier Detection & Treatment
## Detect Outliers (IQR Method)

In [99]:
def iqr_outliers(col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    return col[(col < Q1 - 1.5*IQR) | (col > Q3 + 1.5*IQR)]

for col in num_cols:
    print(col, iqr_outliers(df[col]).shape[0])


Fraud_Ind 7407
Household_Num 0
Policy_Num 0
Age 0
Commute_Discount 0
Claims_Status 0
Claim_Amount 0
Model_Year 0
Initial_Odometer 0
Odometer_At_Incident 0


In [100]:
iso = IsolationForest(contamination=0.02, random_state=42)
outliers = iso.fit_predict(df[num_cols])

df['outlier_flag'] = outliers


In [101]:
for col in num_cols:
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    df[col] = np.clip(df[col], lower, upper)


# STEP 3: Data Type & Format Standardization
## 1.Fix incorrect data types

In [102]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Fraud_Ind                100000 non-null  int64  
 1   Policy_Start_Date        100000 non-null  object 
 2   Policy_Expiry_Date       100000 non-null  object 
 3   Accident_Date            100000 non-null  object 
 4   Claim_Initiated_Date     100000 non-null  object 
 5   Household_Num            100000 non-null  float64
 6   Driver_Num               100000 non-null  object 
 7   Policy_Num               100000 non-null  float64
 8   Claim_Num                100000 non-null  object 
 9   License_Plate            100000 non-null  object 
 10  Drivers_License_Num      100000 non-null  object 
 11  Drivers_License_State    100000 non-null  object 
 12  DL_Expiry_Date           100000 non-null  object 
 13  Date_At_Current_Address  100000 non-null  object 
 14  Age  

##  fraud_ind  should be int 0 or 1

In [58]:
df['Fraud_Ind'] = df['Fraud_Ind'].astype(int)


## Convert dates

In [103]:
date_cols = [
    'Policy_Start_Date',
    'Policy_Expiry_Date',
    'Accident_Date',
    'Claim_Initiated_Date',
    'DL_Expiry_Date',
    'Date_At_Current_Address'
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], format='%m/%d/%Y', errors='coerce')


## Fix ID columns

In [60]:
df['Household_Num'] = df['Household_Num'].astype(str)
df['Policy_Num'] = df['Policy_Num'].astype(str)


## Fix binary flag


In [104]:
df['Commute_Discount'] = df['Commute_Discount'].astype(int)

## Fix status column

In [62]:
df['Claims_Status'] = df['Claims_Status'].astype(str)


In [63]:
df['Police_Report'] = df['Police_Report'].map({'Yes': 1, 'No': 0})


In [121]:
categorical_cols = ['Gender','Claims_Status','Make','Model','Color','Drivers_License_State']
for col in categorical_cols:
    df[col] = df[col].astype('category')


In [122]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 98000 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Fraud_Ind                98000 non-null  int64         
 1   Policy_Start_Date        98000 non-null  datetime64[ns]
 2   Policy_Expiry_Date       98000 non-null  datetime64[ns]
 3   Accident_Date            98000 non-null  datetime64[ns]
 4   Claim_Initiated_Date     98000 non-null  datetime64[ns]
 5   Household_Num            98000 non-null  float64       
 6   Driver_Num               98000 non-null  object        
 7   Policy_Num               98000 non-null  float64       
 8   Claim_Num                98000 non-null  object        
 9   License_Plate            98000 non-null  object        
 10  Drivers_License_Num      98000 non-null  object        
 11  Drivers_License_State    98000 non-null  category      
 12  DL_Expiry_Date           98000 non-nu

# STEP 4: Duplicate & Inconsistent Record Removal

## Duplicate Rows

In [105]:
df.duplicated().sum()



0

## Date Inconsistencies

### Accident before policy start

In [106]:
df = df[df['Accident_Date'] >= df['Policy_Start_Date']]


### Accident after policy expiry

In [107]:
df = df[df['Accident_Date'] <= df['Policy_Expiry_Date']]


### Claim initiated before accident

In [108]:
df = df[df['Claim_Initiated_Date'] >= df['Accident_Date']]


## Odometer Consistency

In [71]:
df = df[df['Odometer_At_Incident'] >= df['Initial_Odometer']]


## Age Validity

In [109]:
df = df[(df['Age'] >= 16) & (df['Age'] <= 100)]


## Claim Amount Validity

In [110]:
df = df[df['Claim_Amount'] > 0]


# STEP 5: Data Leakage Prevention

### Drop Leakage & ID Columns

In [111]:
leakage_cols = [
    'Household_Num',
    'Driver_Num',
    'Policy_Num',
    'Claim_Num',
    'License_Plate',
    'Drivers_License_Num',
    'outlier_flag'
]

df_model = df.drop(columns=leakage_cols)


In [112]:
df.head()

,Fraud_Ind,Policy_Start_Date,Policy_Expiry_Date,Accident_Date,Claim_Initiated_Date,Household_Num,Driver_Num,Policy_Num,Claim_Num,License_Plate,...,Claims_Status,Claim_Amount,Model_Year,Make,Model,Color,Police_Report,Initial_Odometer,Odometer_At_Incident,outlier_flag
0,0,2025-01-27,2025-07-28,2025-03-09,2025-04-08,4.670185e+09,SW059585,355924169.0,DZW11767,MGN62082,...,1,3954,2022,Volkswagen,Jetta,Red,No,87817.0,93539.0,1
1,0,2025-06-01,2025-11-30,2025-06-04,2025-07-05,8.526792e+09,ZO779075,848558311.0,CVG38057,WXK26533,...,4,17917,2011,Toyota,Camry,Grey,Yes,22928.0,30075.0,1
2,0,2025-06-22,2025-12-21,2025-07-04,2025-07-06,4.618821e+09,LG406912,473829217.0,RUO86042,NNL54173,...,3,13884,2015,Audi,A6,Black,No,79831.0,91252.0,1
3,0,2025-01-02,2025-07-03,2025-01-27,2025-02-27,3.846199e+09,NJ146743,116719734.0,WOS87550,THC81673,...,3,4079,2015,BMW,3 Series,White,No,112017.0,117688.0,1
4,0,2024-10-25,2025-04-25,2024-11-19,2024-12-22,9.856771e+09,KX673536,481900450.0,ACJ85041,NKY26268,...,2,3671,2020,Honda,Accord,Silver,No,101637.0,110535.0,1


In [114]:
df.to_pickle('cleaned_data.pkl')


In [116]:
if 'outlier_flag' in df.columns:
    print(df['outlier_flag'].value_counts())


outlier_flag
 1    96035
-1     1965
Name: count, dtype: int64


In [115]:
duplicates = df.duplicated().sum()
print("Number of duplicate rows:", duplicates)


Number of duplicate rows: 0


In [117]:
id_cols = ['Policy_Num','Claim_Num','Driver_Num']
for col in id_cols:
    print(col, "unique:", df[col].nunique(), "vs total rows:", len(df))


Policy_Num unique: 96045 vs total rows: 98000
Claim_Num unique: 97995 vs total rows: 98000
Driver_Num unique: 97992 vs total rows: 98000


In [118]:
# Accident_Date should not be after Claim_Initiated_Date
leak_check = df[df['Accident_Date'] > df['Claim_Initiated_Date']]
print("Rows violating date order (should be 0):", len(leak_check))


Rows violating date order (should be 0): 0


In [76]:
df.shape


(98000, 27)